# Scraping Web Page with jokes to collect data for dataset

In [ ]:
# crash colab to get more RAM
#!kill -9 -1

In [ ]:
!nvidia-smi

Fri Dec 24 23:49:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P8    30W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!rm -rf /usr/local/cuda
!ln -s /usr/local/cuda-10.1 /usr/local/cuda

In [ ]:
%%bash
export LD_LIBRARY_PATH=/usr/lib/

In [ ]:
%%capture
!apt-get install clang-9 llvm-9 llvm-9-dev llvm-9-tools

In [ ]:
import subprocess

CUDA_version = [s for s in subprocess.check_output(["nvcc", "--version"]).decode("UTF-8").split(", ") if s.startswith("release")][0].split(" ")[-1]
print("CUDA version:", CUDA_version)

if CUDA_version == "10.0":
    torch_version_suffix = "+cu100"
elif CUDA_version == "10.1":
    torch_version_suffix = "+cu101"
elif CUDA_version == "10.2":
    torch_version_suffix = ""
else:
    torch_version_suffix = "+cu110"

CUDA version: 10.1


In [ ]:
%%capture
!pip install torch==1.7.1{torch_version_suffix} torchvision==0.8.2{torch_version_suffix} torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
!pip3 install transformers==3.5.0

     |████████████████████████████████| 1.3 MB 4.0 MB/s 
     |████████████████████████████████| 895 kB 41.9 MB/s 
     |████████████████████████████████| 1.1 MB 33.1 MB/s 
     |████████████████████████████████| 2.9 MB 37.3 MB/s 


In [ ]:
%%capture
!pip install scrapy

In [ ]:
!git clone  https://github.com/sberbank-ai/ru-gpts

Cloning into 'ru-gpts'...
remote: Enumerating objects: 683, done.
remote: Counting objects: 100% (178/178), done.
remote: Compressing objects: 100% (94/94), done.
remote: Total 683 (delta 110), reused 141 (delta 83), pack-reused 505
Receiving objects: 100% (683/683), 413.81 KiB | 3.02 MiB/s, done.
Resolving deltas: 100% (410/410), done.


In [ ]:
import pandas as pd 
import tensorflow as tf

import numpy as np
import os
import time
import torch

from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
!scrapy startproject jokes

New Scrapy project 'jokes', using template directory '/usr/local/lib/python3.7/dist-packages/scrapy/templates/project', created in:
    /content/jokes

You can start your first spider with:
    cd jokes
    scrapy genspider example example.com


In [ ]:
!scrapy genspider jokeSpider 'https://vse-shutochki.ru/anekdoty/'

Created spider 'jokeSpider' using template 'basic' 


In [ ]:
# Scraping WEB-SITE
# Создаем собственный датасет с шутками с сайта 'https://vse-shutochki.ru/anekdoty/' (обрабатываем 2 с небольшим тысячи страниц сайта)
!scrapy runspider jokeSpider.py -o "scraped_data.csv" -L INFO
# IMPORTANT: replace jokeSpider.py
##################################
#Код для jokeSpider.py
# import scrapy

# class JokespiderSpider(scrapy.Spider):
#     name = 'jokeSpider'
#     start_urls = ['https://vse-shutochki.ru/anekdoty//']

#     def parse(self, response):
#         joke_raw = response.xpath('//div[@class="post"]')
#         for post in joke_raw:
#             text = ' '.join(post.xpath('text()').extract()).rstrip().lstrip()
#             if text:
#                 yield {'joke': text}      
#         next_page = response.xpath(".//a[@class='btn btn-primary btn-large']/@href")[0]
#         if next_page.get() != '/anekdoty/2069':
#             yield response.follow(next_page, self.parse)
###################################

# Import already scrapped web site
from google.colab import drive
drive.mount('/content/gdrive/', force_remount=True)
%cd '/content/gdrive/My Drive/nlp_data/'

Mounted at /content/gdrive/
/content/gdrive/My Drive/nlp_data


In [ ]:
file = 'scraped_data.csv'
data = pd.read_csv(file)

In [ ]:
data.shape

(61405, 1)

In [ ]:
data.head()

,joke
0,Старик Хоттабыч просыпается утром со страшного...
1,"- Сaня, а скaжи чeстно, чтo ты почувствoвал, к..."
2,Средневековая Англия. У дороги сидит нищий. Из...
3,"Пришел домой пораньше, в комнате незнакомый му..."
4,"- Папа, - спрашивает сын, - а правда, что наст..."


In [ ]:
data_array = [joke for joke in data['joke']]

In [ ]:
data_array[4]

'- Папа, - спрашивает сын, - а правда, что настоящий мужчина должен оставаться спокойным в любой ситуации?  - Правда, сынок.  - Тогда тебе сначала показать мой дневник или новое платье мамы?'

In [ ]:
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

In [ ]:
def load_tokenizer_and_model(model_name_or_path):
  return GPT2Tokenizer.from_pretrained(model_name_or_path), GPT2LMHeadModel.from_pretrained(model_name_or_path).cuda()

def generate(
    model, tok, text,
    do_sample=True, max_length=500, repetition_penalty=5.0,
    top_k=5, top_p=0.95, temperature=1,
    num_beams=None,
    no_repeat_ngram_size=3
    ):
  input_ids = tok.encode(text, return_tensors="pt").cuda()
  out = model.generate(
      input_ids.cuda(),
      max_length=max_length,
      repetition_penalty=repetition_penalty,
      do_sample=do_sample,
      top_k=top_k, top_p=top_p, temperature=temperature,
      num_beams=num_beams, no_repeat_ngram_size=no_repeat_ngram_size
      )
  return list(map(tok.decode, out))

In [ ]:
tok, model = load_tokenizer_and_model("sberbank-ai/rugpt3large_based_on_gpt2")
generated = generate(model, tok, data_array[0], num_beams=10)

Downloading:   0%|          | 0.00/1.71M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.27M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/609 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.14G [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
generated
# RESULT
#['Старик Хоттабыч просыпается утром со страшного похмелья. Тянется рукой к бороде, а бороды нет, ни одного волоска. Хвать за голову на голове тоже нет, он рукой подмышку и там пусто, осторожно оттягивает резинку трусов: - Ну слава богу хоть на пиво осталось! А то я уж думал, не доживу до утра... И тут же спохватывается: - Что ж это я так опохмеляюсь-то? Да еще в такую рань?! Встаю с кровати, иду умываться холодной водой из умывальника. Прохладная вода освежает тело, но голова все равно трещит как переспелая груша. Смотрю на себя в большое круглое зеркало над умывальником. Лицо опухшее, глаза красные, нос распухший, волосы всклокоченные, щетина торчит во все стороны. Одеваюсь, выхожу на улицу. Свежий утренний ветерок приятно холодит разгоряченное лицо. Подхожу к киоску "Союзпечать", покупаю свежий номер журнала "Огонек". Заворачиваю его в газетную бумагу и прячу в задний карман джинсов. Возвращаюсь домой. По дороге захожу в парикмахерскую. Парикмахер спрашивает меня: - Сколько вам лет? Я отвечаю: - Девятнадцать. Он говорит: - Вы очень молодо выглядите для своего возраста. У вас есть какие-нибудь пожелания по поводу прически или макияжа? Я говорю: - Мне бы хотелось подстричься покороче да покрасивее. На что он мне отвечает: - Это можно будет сделать только после того, как у вас отрастут ваши собственные волосы. После стрижки они должны быть такими же длинными, как те, которыми вы сейчас пользуетесь. Когда я вернулся домой, жена уже накрывала на стол. Увидев меня, она радостно воскликнула: - Дорогой, наконец-то ты пришел! Как хорошо, что мы тебя дождались! Ты даже не представляешь себе, какой сюрприз нам сегодня преподнесла твоя новая прическа! Она достала из шкафчика расческу и начала приводить мою прическу в порядок. Через несколько минут передо мной стоял совершенно другой человек. Волосы мои были коротко острижены, брови аккуратно подкрашены тушью, губы накрашены ярко-красной помадой. Выглядел я теперь гораздо моложе своих двадцати пяти лет. Жена смотрела на меня с восхищением. Мы сели ужинать.']

['Старик Хоттабыч просыпается утром со страшного похмелья. Тянется рукой к бороде, а бороды нет, ни одного волоска. Хвать за голову на голове тоже нет, он рукой подмышку и там пусто, осторожно оттягивает резинку трусов: - Ну слава богу хоть на пиво осталось! А то я уж думал, не доживу до утра... И тут же спохватывается: - Что ж это я так опохмеляюсь-то? Да еще в такую рань?! Встаю с кровати, иду умываться холодной водой из умывальника. Прохладная вода освежает тело, но голова все равно трещит как переспелая груша. Смотрю на себя в большое круглое зеркало над умывальником. Лицо опухшее, глаза красные, нос распухший, волосы всклокоченные, щетина торчит во все стороны. Одеваюсь, выхожу на улицу. Свежий утренний ветерок приятно холодит разгоряченное лицо. Подхожу к киоску "Союзпечать", покупаю свежий номер журнала "Огонек". Заворачиваю его в газетную бумагу и прячу в задний карман джинсов. Возвращаюсь домой. По дороге захожу в парикмахерскую. Парикмахер спрашивает меня: - Сколько вам лет? 

In [ ]:
generated_2 = generate(model, tok, data_array[6], num_beams=10)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
generated_2
#RESULT
#['Биробиджан. Ночь. Супруги в постели. — Яша, ты таки накажешь меня сегодня ночью? — Брось, Софочка, ты ни в чем не виновата...\n— А что я должна была сделать? Ты же знаешь, как я к тебе отношусь...\n— Не знаю, но если бы знал, то наказал бы обязательно!\nСофочка проснулась от того, что кто-то настойчиво тряс ее за плечо. Открыв глаза, она увидела перед собой улыбающегося Яшу.\n— Вставай, соня, а то опоздаем на поезд.\nЯша помог ей одеться и они вместе вышли из дома. На улице было уже совсем темно. Яркий свет уличных фонарей освещал пустынные улицы Биробиджана. Они быстро шли по направлению к железнодорожному вокзалу. Навстречу им попалась группа молодых людей с гитарой. Молодые люди весело о чем-то переговаривались между собой. Вдруг один из них запел какую-то веселую песенку. Его подхватили все остальные. Песня звучала так громко, что заглушала шум проезжающих мимо автомобилей. Когда песня закончилась, молодой человек подошел к Яше и протянул ему гитару:\n— Давай сыграем для тебя одну нашу новую песню. Она называется «Птица счастья завтрашнего дня».\nМолодой человек был очень симпатичным молодым человеком лет двадцати пяти или около того. У него были светло-каштановые волосы, карие глаза и смуглая загорелая кожа. Одет он был в джинсы и белую футболку без рукавов. Смуглое лицо молодого человека выражало дружелюбие и жизнерадостность. Молодой человек взял у Яши гитару и заиграл тихую грустную мелодию. Все присутствующие стали подпевать певцу. Через некоторое время мелодия стала еще более печальной. Певец пел о том, что скоро наступит день, когда птицы будут петь радостные песни об этом прекрасном дне. После этой песни молодые люди попрощались со своим кумиром и пошли своей дорогой. Вскоре их голоса затихли где-то вдали.<s>\nНажимая кнопку "Зарегистрироваться", вы подтверждаете свое согласие с условиями предоставления услуг (Политикой конфиденциальности).\nОбращаем ваше внимание на то, что данный интернет-сайт носит исключительно информационный характер и ни при каких условиях не является публичной офертой, определяемой положениями Статьи 437 Гражданского кодекса Российской Федерации. Для получения подробной информации о наличии и стоимости указанных товаров']

['Биробиджан. Ночь. Супруги в постели. — Яша, ты таки накажешь меня сегодня ночью? — Брось, Софочка, ты ни в чем не виновата...\n— А что я должна была сделать? Ты же знаешь, как я к тебе отношусь...\n— Не знаю, но если бы знал, то наказал бы обязательно!\nСофочка проснулась от того, что кто-то настойчиво тряс ее за плечо. Открыв глаза, она увидела перед собой улыбающегося Яшу.\n— Вставай, соня, а то опоздаем на поезд.\nЯша помог ей одеться и они вместе вышли из дома. На улице было уже совсем темно. Яркий свет уличных фонарей освещал пустынные улицы Биробиджана. Они быстро шли по направлению к железнодорожному вокзалу. Навстречу им попалась группа молодых людей с гитарой. Молодые люди весело о чем-то переговаривались между собой. Вдруг один из них запел какую-то веселую песенку. Его подхватили все остальные. Песня звучала так громко, что заглушала шум проезжающих мимо автомобилей. Когда песня закончилась, молодой человек подошел к Яше и протянул ему гитару:\n— Давай сыграем для тебя одн

In [ ]:
generated_3 = generate(model, tok, data_array[2], num_beams=10)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


In [ ]:
generated_3
# RESULT
# ['Средневековая Англия. У дороги сидит нищий. Из леса выходит Робин Гуд и бросает к ногам нищего мешок золота со словами:. — Я Робин Гуд, отбираю у богатых и отдаю бедным!  Нищий ликует: --\u200bНаконец-то и мне улыбнулось счастье! Я теперь сказочно богат! Робин Гуд протыкает нищего мечом, забирает золото и говорит: — Я Робин Гуд, отбираю у богатых и отдаю бедным!<s>\nЧто делать если не знаешь как поступить в данной ситуации?\nПопробуйте поговорить с ним на эту тему. Если он вас любит, то поймёт что вы ему небезразличны. А если нет, то это его проблемы. Не стоит из-за таких пустяков портить себе жизнь.<s>\nПомогите пожалуйста решить задачу по геометрии за 7 класс!!!\n1) 3*2=5 2) 4*3=6 3) 5*4=8 4) 6*7=12 8) 9*9=16 10) 15*15=40 11) 20*20=60 12) 30*30=80 13) 40*40=120 14) 50*50=150 15) 60*60=180 16) 70*70=300 17) 80*80=400 18) 90*90=500 19) 100*100=600 20) 200*200=700 21) 300*300=800 22) 400*400=1000 23) 600*600=1400 24) 800*800=1700 25) 1600*1600=2000 26) 1800*1800=3500 27) 2000*1900=4500 28) 3500*2000=7500 29) 4500*3600=10 000 30) 2500*2500=11 000 31) 3000*2700=6500 32) 4000*2800=8500 33) 5500*3800=110 000 34) 6500*3900=130 000 35) 7500*4100=160 000 36) 8500*4200=200 000 37) 160*8500=250 000 38) 250*8400=360 000 39) 360*8600= 500 000 40) 550*8700= 700 000 41) 750*8800= 1200 000 42) 900*8900=1300 43) 1000*9800= 1500 44) 1900*9900= 1950 45)']

['Средневековая Англия. У дороги сидит нищий. Из леса выходит Робин Гуд и бросает к ногам нищего мешок золота со словами:. — Я Робин Гуд, отбираю у богатых и отдаю бедным!  Нищий ликует: --\u200bНаконец-то и мне улыбнулось счастье! Я теперь сказочно богат! Робин Гуд протыкает нищего мечом, забирает золото и говорит: — Я Робин Гуд, отбираю у богатых и отдаю бедным!<s>\nЧто делать если не знаешь как поступить в данной ситуации?\nПопробуйте поговорить с ним на эту тему. Если он вас любит, то поймёт что вы ему небезразличны. А если нет, то это его проблемы. Не стоит из-за таких пустяков портить себе жизнь.<s>\nПомогите пожалуйста решить задачу по геометрии за 7 класс!!!\n1) 3*2=5 2) 4*3=6 3) 5*4=8 4) 6*7=12 8) 9*9=16 10) 15*15=40 11) 20*20=60 12) 30*30=80 13) 40*40=120 14) 50*50=150 15) 60*60=180 16) 70*70=300 17) 80*80=400 18) 90*90=500 19) 100*100=600 20) 200*200=700 21) 300*300=800 22) 400*400=1000 23) 600*600=1400 24) 800*800=1700 25) 1600*1600=2000 26) 1800*1800=3500 27) 2000*1900=450